<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Use FEniCSx to solve an reaction-diffusion equation
$$-\epsilon^2 \Delta(u) + u = \sqrt{3/2+x-y} \in (0,1)^2$$
with homogeneous Dirichlet boundary conditions.

* Standard P1 Galerkin FEM.
* Shishkin mesh
* Direct solver

**WARNING** May fail for large $N=2^8$ or very small $\varepsilon=10^{-8}$

This version is an example for Ram Shiromani by Niall Madden, Nov 2023.

In [1]:
from mpi4py import MPI
from dolfinx import mesh

In [2]:
import dolfinx
import dolfinx.io
from dolfinx.fem import FunctionSpace
from dolfinx import fem, log, fem, io, mesh, plot
from dolfinx.mesh import CellType
from dolfinx.fem.petsc import LinearProblem

import ufl
from ufl import ds, dx, grad, inner

from petsc4py import PETSc
from petsc4py.PETSc import ScalarType
import numpy as np
#from fenicsx_plotly import plot
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
import os
#log.set_log_level(log.LogLevel.INFO)

Set `SlowPlots=True` to see output, but you need vtk packages installed. I prefer to output to a `.xdmk` file, and view that in paraview.

In [3]:
ShowPlots = False # don't plot for big N

In [4]:
def mpi_print(s):
    print(f"{comm.rank}/{comm.size}: {s}")

In [5]:
import time
Start = time.time()
comm = MPI.COMM_WORLD

## Functions for making a mesh

In [6]:
def Shishkin_Mesh_1D_2(tau, N, layer_frac):
    """ Generate a 1D piecewise uniform Shishkin mesh,
    with N*layer_frac points in [0,tau] and [1-tau,1], with
    and N*(1-2*layer_frac) in [tau,1-tau]. 
    returns a numpy array
    WARNING: should check inputs make sense. By NM 2022
    """

    N_layer = int(layer_frac*N)
    N_interior = N - 2*N_layer

    xl = np.linspace(0,tau,N_layer+1)
    xc = np.linspace(tau,1-tau, N_interior+1)
    xr = np.linspace(1-tau,1,N_layer+1)    
    x_s = np.concatenate([xl,xc[1:],xr[1:]])

    return x_s

In [7]:
def Tensor_Product_Mesh_2D(x_p, y_p):
    """ Generate a 2D tensor product mesh based on the
     one-dimensional meshes mesh_x and mesh_y
    Input: 2 numpy arrays, with M and M enries
    output: a numpy array with M*N entries. By NM 2022.
    """

    (X,Y) = np.meshgrid(x_p,y_p)
    X_q = np.vstack( (X.flatten(), Y.flatten()) ).T

    #Nx = len(x_p)-1
    #Ny = len(y_p)-1
    #X_p = np.empty([(Nx+1)*(Ny+1),2])
    #k=0
    #for yi in y_p:
    #    for xi in x_p:
    #        X_p[k,:] = [xi,yi]
    #        k+=1
    return X_q

## PDE data
PDE Data for $$-\epsilon^2 \Delta u + u = F(x,y)$$

In [8]:
epsilon = 1.0e-3
def F(x):
    return np.sqrt(1.5 + x[0] - x[1])

## Discretization data

In [9]:
N = 2**5
tau = 2*epsilon*np.log(N)

In [10]:
x1 = Shishkin_Mesh_1D_2(tau, N, 0.25)
MeshPoints = Tensor_Product_Mesh_2D(x1,x1)
Triangulation = Delaunay(MeshPoints)

In [11]:
#Triangulation.simplices.astype(np.int64)

In [12]:
if ShowPlots:
    plt.triplot(Triangulation.points[:,0], Triangulation.points[:,1], Triangulation.simplices.astype(np.int64))

In [13]:
ufl_P1 = ufl.Mesh(ufl.VectorElement("Lagrange", ufl.triangle, 1))
msh = mesh.create_mesh(comm=MPI.COMM_WORLD, cells=Triangulation.simplices, x=Triangulation.points, \
                       domain=ufl_P1)

In [14]:
V = fem.FunctionSpace(msh, ("Lagrange", 1))
msh.topology.create_connectivity(msh.topology.dim-1, msh.topology.dim)
boundary_facets = mesh.exterior_facet_indices(msh.topology)
boundary_dofs = fem.locate_dofs_topological(V, msh.topology.dim-1, boundary_facets)

dof_coordinates = V.tabulate_dof_coordinates()[boundary_dofs]
#print(dof_coordinates)
u_bc = dolfinx.fem.Function(V)
bc = dolfinx.fem.dirichletbc(u_bc, boundary_dofs)

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)

In [15]:
f = fem.Function(V)
f.interpolate(lambda x: F(x))


In [16]:
a = ufl.dot((epsilon**2.0)*ufl.grad(u), ufl.grad(v)) * ufl.dx \
    + ufl.dot(u, v) * ufl.dx 
L = f * v * dx

In [17]:
problem = LinearProblem(a, L, bcs=[bc])
uN = problem.solve()

In [18]:
viewer = PETSc.Viewer().createASCII("solver_output.txt")
problem.solver.view(viewer)
solver_output = open("solver_output.txt", "r")
#for line in solver_output.readlines():
#    print(line)

In [19]:
elapsed_time = time.time() - Start
if comm.rank == 0:
    mpi_print(f"Done. N={N}, epsilon={epsilon}, time = {elapsed_time}")

0/1: Done. N=32, epsilon=0.001, time = 0.17171859741210938


In [20]:
with io.XDMFFile(msh.comm, "2DRD.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(uN)

In [21]:
if ShowPlots:
    import pyvista
    cells, types, x = plot.create_vtk_mesh(V)
    grid = pyvista.UnstructuredGrid(cells, types, x)
    grid.point_data["u"] = uN.x.array.real
    grid.set_active_scalars("u")
    plotter = pyvista.Plotter()
    plotter.add_mesh(grid, show_edges=True)
    warped = grid.warp_by_scalar()
    plotter.add_mesh(warped)
    plotter.show()